# HDAT Capstone Project

## Research Question 1 - Mortality prediction in the ICU:

#### Task - The task is to build a predictive algorithm using the techniques we learned in this course
#### Objective - To assess the role of machine learning algorithms for predicting mortality by using the MIMIC-II dataset
#### Question - Is it possible to accurately predict mortality based on data from the first 24 hours in ICU?
#### Study population - MIMIC-II dataset

## Research Question 2  - Weekend Effect in the ICU

#### Task - The task is to investigate whether admission to ICU at the weekend increases the risk of ICU mortality 
#### Objective - To develop a statistical model to estimate the effect of weekend admission to ICU on the risk of mortality. 
#### Question - Does admission to ICU over the weekend increase the risk of mortality? 
#### Study population - MIMIC-II dataset

Notes about the datsets:

1. Incorrect values - MIMIC-II was not collected for research and is a combination of two different electronic medical record systems (CareVue and Metavision). This increase the likelihood of inaccuracies in data entry and extraction.

2. Missing data/sparseness: there is variation in the information recorded between patients due to different uses of the EMR (e.g. use of a separate system for recording lab results, or medications) across time, and the data being collected for clinical relevance rather than research.

All patients have a unique identifying ID (subject_id), a hospital stay ID (hadm_id) and an ICU stay ID (icustay_id). These IDs can be used to identify readmissions to hospital and ICU.

# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) # show all columns
import numpy as np 
import seaborn as sns
sns.set_style("darkgrid") # sets seaborn plot style guide
import matplotlib.pyplot as plt 


In [2]:
# Define function to examine percentage of missing data by column
def missing_data(df):
    for column in df.columns:
        print(f'Column {column}', f'has {100 * sum(df[column].isnull())/len(df):.2f}% missing data')
        print()
    return

## Loading in datasets and merging 

In [3]:
# load in pt_stay_hr as the building block / master table
pt_stay_hr = pd.read_csv('mimic_data/pt_stay_hr.csv')
pt_stay_hr.head()

,icustay_id,hadm_id,subject_id,intime,outtime,starttime,endtime,hr,dy
0,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 19:06:12,2181-11-24 20:06:12,-24.0,0.0
1,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 20:06:12,2181-11-24 21:06:12,-23.0,0.0
2,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 21:06:12,2181-11-24 22:06:12,-22.0,0.0
3,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 22:06:12,2181-11-24 23:06:12,-21.0,0.0
4,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 23:06:12,2181-11-25 00:06:12,-20.0,0.0


In [4]:
patients = pd.read_csv('mimic_data/patients.csv') # https://mimic.physionet.org/mimictables/patients/
# Table purpose: Defines each SUBJECT_ID in the database, i.e. defines a single patient
# Links to: ADMISSIONS on SUBJECT_ID, ICUSTAYS on SUBJECT_ID
patients.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [5]:
# Select only columns of interest from patients
patients = patients[['subject_id','gender','dob']]
patients.head()

,subject_id,gender,dob
0,249,F,2075-03-13 00:00:00
1,250,F,2164-12-27 00:00:00
2,251,M,2090-03-15 00:00:00
3,252,M,2078-03-06 00:00:00
4,253,F,2089-11-26 00:00:00


In [6]:
# Merge pt_stay_hr and patients to master table on subject_id

master = pd.merge(pt_stay_hr, patients, on='subject_id')
master.head()
# intime + outtime = ICU in and out times
# hr starts from -24 = 24 hrs before admission
# dy days in ICU
# starttime and endtime = start and end of each hr interval

,icustay_id,hadm_id,subject_id,intime,outtime,starttime,endtime,hr,dy,gender,dob
0,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 19:06:12,2181-11-24 20:06:12,-24.0,0.0,F,2120-10-31 00:00:00
1,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 20:06:12,2181-11-24 21:06:12,-23.0,0.0,F,2120-10-31 00:00:00
2,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 21:06:12,2181-11-24 22:06:12,-22.0,0.0,F,2120-10-31 00:00:00
3,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 22:06:12,2181-11-24 23:06:12,-21.0,0.0,F,2120-10-31 00:00:00
4,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 23:06:12,2181-11-25 00:06:12,-20.0,0.0,F,2120-10-31 00:00:00


In [7]:
missing_data(master)

Column icustay_id has 0.00% missing data

Column hadm_id has 0.00% missing data

Column subject_id has 0.00% missing data

Column intime has 0.00% missing data

Column outtime has 0.00% missing data

Column starttime has 0.00% missing data

Column endtime has 0.00% missing data

Column hr has 0.00% missing data

Column dy has 0.04% missing data

Column gender has 0.00% missing data

Column dob has 0.00% missing data



In [8]:
# Load pt_icu_outcome dataset
pt_icu_outcome = pd.read_csv('mimic_data/pt_icu_outcome.csv')
pt_icu_outcome.head()

,row_id,subject_id,dob,hadm_id,admittime,dischtime,icustay_id,age_years,intime,outtime,los,hosp_deathtime,icu_expire_flag,hospital_expire_flag,dod,expire_flag,ttd_days
0,1,2,2138-07-17 00:00:00,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,243653,0.0,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918,NaN,0,0.0,NaN,0,NaN
1,2,3,2025-04-11 00:00:00,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,211552,76.0,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,NaN,0,0.0,2102-06-14 00:00:00,1,236.0
2,3,4,2143-05-12 00:00:00,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,294638,47.0,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,NaN,0,0.0,NaN,0,NaN
3,4,5,2103-02-02 00:00:00,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,214757,0.0,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844,NaN,0,0.0,NaN,0,NaN
4,5,6,2109-06-21 00:00:00,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,228232,65.0,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,NaN,0,0.0,NaN,0,NaN


In [9]:
# Selecting only columns of interest from pt_icu
pt_icu_outcome = pt_icu_outcome[['icustay_id','age_years','los','icu_expire_flag', 'ttd_days']]
pt_icu_outcome.head()

,icustay_id,age_years,los,icu_expire_flag,ttd_days
0,243653,0.0,0.0918,0,NaN
1,211552,76.0,6.0646,0,236.0
2,294638,47.0,1.6785,0,NaN
3,214757,0.0,0.0844,0,NaN
4,228232,65.0,3.6729,0,NaN


In [10]:
# Left join the master table with our selected variables from pt_icu_outcome on the icustay_id
master = pd.merge(master, pt_icu_outcome, on='icustay_id', how='left')
master.head()

,icustay_id,hadm_id,subject_id,intime,outtime,starttime,endtime,hr,dy,gender,dob,age_years,los,icu_expire_flag,ttd_days
0,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 19:06:12,2181-11-24 20:06:12,-24.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0
1,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 20:06:12,2181-11-24 21:06:12,-23.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0
2,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 21:06:12,2181-11-24 22:06:12,-22.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0
3,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 22:06:12,2181-11-24 23:06:12,-21.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0
4,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 23:06:12,2181-11-25 00:06:12,-20.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0


In [11]:
missing_data(master)

Column icustay_id has 0.00% missing data

Column hadm_id has 0.00% missing data

Column subject_id has 0.00% missing data

Column intime has 0.00% missing data

Column outtime has 0.00% missing data

Column starttime has 0.00% missing data

Column endtime has 0.00% missing data

Column hr has 0.00% missing data

Column dy has 0.04% missing data

Column gender has 0.00% missing data

Column dob has 0.00% missing data

Column age_years has 0.00% missing data

Column los has 0.00% missing data

Column icu_expire_flag has 0.00% missing data

Column ttd_days has 59.25% missing data



In [12]:
# Load in gcs_hourly --> Glasgow Coma Score
gcs_hourly = pd.read_csv('mimic_data/gcs_hourly.csv')
gcs_hourly.head()

,icustay_id,hr,gcs,gcseyes,gcsmotor,gcsverbal,endotrachflag
0,200001,0,15,4.0,6.0,5.0,0
1,200001,4,15,4.0,6.0,5.0,0
2,200001,11,15,4.0,6.0,5.0,0
3,200001,13,15,4.0,6.0,5.0,0
4,200001,16,14,3.0,6.0,5.0,0


In [13]:
# Select variables of interest --> gcs (overall score) and endotrachflag (endotracheal tube) + hr and icustay_id for linkage
gcs_hourly = gcs_hourly[['icustay_id', 'hr', 'gcs', 'endotrachflag']]
gcs_hourly.head()

,icustay_id,hr,gcs,endotrachflag
0,200001,0,15,0
1,200001,4,15,0
2,200001,11,15,0
3,200001,13,15,0
4,200001,16,14,0


In [14]:
master = pd.merge(master, gcs_hourly, how='left', on=['icustay_id','hr'])
master.head()

,icustay_id,hadm_id,subject_id,intime,outtime,starttime,endtime,hr,dy,gender,dob,age_years,los,icu_expire_flag,ttd_days,gcs,endotrachflag
0,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 19:06:12,2181-11-24 20:06:12,-24.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN
1,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 20:06:12,2181-11-24 21:06:12,-23.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN
2,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 21:06:12,2181-11-24 22:06:12,-22.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN
3,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 22:06:12,2181-11-24 23:06:12,-21.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN
4,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 23:06:12,2181-11-25 00:06:12,-20.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN


In [15]:
missing_data(master)

Column icustay_id has 0.00% missing data

Column hadm_id has 0.00% missing data

Column subject_id has 0.00% missing data

Column intime has 0.00% missing data

Column outtime has 0.00% missing data

Column starttime has 0.00% missing data

Column endtime has 0.00% missing data

Column hr has 0.00% missing data

Column dy has 0.04% missing data

Column gender has 0.00% missing data

Column dob has 0.00% missing data

Column age_years has 0.00% missing data

Column los has 0.00% missing data

Column icu_expire_flag has 0.00% missing data

Column ttd_days has 59.25% missing data

Column gcs has 82.80% missing data

Column endotrachflag has 82.80% missing data



In [16]:
# load in vitals_hourly dataset
vitals_hourly = pd.read_csv('mimic_data/vitals_hourly.csv')
vitals_hourly.head()

,icustay_id,hr,spo2,fio2,temperature,resprate,heartrate,sysbp,diasbp,glucose,meanarterialpressure
0,200001,1,98.0,NaN,NaN,18.0,108.0,113.0,68.0,NaN,79.0
1,200001,2,98.0,NaN,NaN,27.0,110.0,116.0,68.0,118.0,79.0
2,200001,3,99.8,NaN,37.666667,21.0,102.0,102.0,61.0,NaN,71.0
3,200001,4,94.0,NaN,NaN,19.0,108.0,103.0,58.0,NaN,69.0
4,200001,5,100.0,35.0,NaN,28.0,104.0,106.0,62.0,NaN,73.0


In [17]:
# Extract variables of interest --> spo2, heartrate, meanarterialpressure + icustay_id and hr for linkage
# spo2 chosen as there is little NA values over fio2 --> constantly monitored, low values are a sign of a serious failure in the respiratory system
# heartrate can be an estimate of cardiovascular function and overall physiological function
# mean arterial pressure (MAP) incorporates systolic and diastolic blood pressures into one measure and hence is quite useful for machine learning

vitals_hourly = vitals_hourly[['icustay_id', 'hr', 'spo2', 'heartrate', 'meanarterialpressure']]
vitals_hourly.head()

,icustay_id,hr,spo2,heartrate,meanarterialpressure
0,200001,1,98.0,108.0,79.0
1,200001,2,98.0,110.0,79.0
2,200001,3,99.8,102.0,71.0
3,200001,4,94.0,108.0,69.0
4,200001,5,100.0,104.0,73.0


In [18]:
# merge with master
master = pd.merge(master, vitals_hourly, how='left', on=['icustay_id','hr'])
master.head()

,icustay_id,hadm_id,subject_id,intime,outtime,starttime,endtime,hr,dy,gender,dob,age_years,los,icu_expire_flag,ttd_days,gcs,endotrachflag,spo2,heartrate,meanarterialpressure
0,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 19:06:12,2181-11-24 20:06:12,-24.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN
1,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 20:06:12,2181-11-24 21:06:12,-23.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN
2,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 21:06:12,2181-11-24 22:06:12,-22.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN
3,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 22:06:12,2181-11-24 23:06:12,-21.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN
4,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 23:06:12,2181-11-25 00:06:12,-20.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN


In [19]:
missing_data(master)

Column icustay_id has 0.00% missing data

Column hadm_id has 0.00% missing data

Column subject_id has 0.00% missing data

Column intime has 0.00% missing data

Column outtime has 0.00% missing data

Column starttime has 0.00% missing data

Column endtime has 0.00% missing data

Column hr has 0.00% missing data

Column dy has 0.04% missing data

Column gender has 0.00% missing data

Column dob has 0.00% missing data

Column age_years has 0.00% missing data

Column los has 0.00% missing data

Column icu_expire_flag has 0.00% missing data

Column ttd_days has 59.25% missing data

Column gcs has 82.80% missing data

Column endotrachflag has 82.80% missing data

Column spo2 has 39.45% missing data

Column heartrate has 27.66% missing data

Column meanarterialpressure has 47.07% missing data



In [20]:
# load in urine output 
output_hourly = pd.read_csv('mimic_data/output_hourly.csv')
output_hourly.head()

,icustay_id,hr,urineoutput
0,200001,2,250.0
1,200001,25,60.0
2,200001,62,50.0
3,200003,0,230.0
4,200003,2,0.0


In [21]:
# merge with master
master = pd.merge(master, output_hourly, how='left', on=['icustay_id','hr'])
master.head()

,icustay_id,hadm_id,subject_id,intime,outtime,starttime,endtime,hr,dy,gender,dob,age_years,los,icu_expire_flag,ttd_days,gcs,endotrachflag,spo2,heartrate,meanarterialpressure,urineoutput
0,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 19:06:12,2181-11-24 20:06:12,-24.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN
1,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 20:06:12,2181-11-24 21:06:12,-23.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN
2,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 21:06:12,2181-11-24 22:06:12,-22.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN
3,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 22:06:12,2181-11-24 23:06:12,-21.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN
4,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 23:06:12,2181-11-25 00:06:12,-20.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# load in blood culture dataset
blood_culture = pd.read_csv('mimic_data/bloodculture.csv')
# keeping relevent rows
blood_culture = blood_culture[['icustay_id', 'dy', 'hr', 'positiveculture']]
blood_culture.head()

,icustay_id,dy,hr,positiveculture
0,217870.0,4.0,71.0,1
1,217870.0,4.0,71.0,1
2,217870.0,4.0,71.0,1
3,217870.0,4.0,71.0,1
4,217870.0,5.0,93.0,0


In [23]:
# merge blood culture with master
master = pd.merge(master, blood_culture, how='left')
master.head(100)

,icustay_id,hadm_id,subject_id,intime,outtime,starttime,endtime,hr,dy,gender,dob,age_years,los,icu_expire_flag,ttd_days,gcs,endotrachflag,spo2,heartrate,meanarterialpressure,urineoutput,positiveculture
0,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 19:06:12,2181-11-24 20:06:12,-24.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 20:06:12,2181-11-24 21:06:12,-23.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 21:06:12,2181-11-24 22:06:12,-22.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 22:06:12,2181-11-24 23:06:12,-21.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 23:06:12,2181-11-25 00:06:12,-20.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-28 18:06:12,2181-11-28 19:06:12,71.0,3.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,100.0,89.0,84.0,NaN,NaN
96,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-28 19:06:12,2181-11-28 20:06:12,72.0,4.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,15.0,0.0,98.2,86.0,58.0,NaN,NaN
97,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-28 20:06:12,2181-11-28 21:06:12,73.0,4.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-28 21:06:12,2181-11-28 22:06:12,74.0,4.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# load in antibiotics dataset
antibiotics = pd.read_csv('mimic_data/antibiotics.csv')
antibiotics.head()

,icustay_id,starttime,endtime,amount,amountuom,rate,rateuom,ordercategoryname,patientweight,totalamount,totalamountuom,statusdescription,label,abbreviation,antibiotic,dbsource
0,200033.0,2198-08-11 14:40:00,2198-08-11 14:41:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),74.0,100.0,ml,FinishedRunning,Piperacillin/Tazobactam (Zosyn),Piperacillin/Tazobactam (Zosyn),1,metavision
1,200033.0,2198-08-11 15:36:00,2198-08-11 15:37:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),74.0,200.0,ml,FinishedRunning,Ciprofloxacin,Ciprofloxacin,1,metavision
2,200033.0,2198-08-11 22:05:00,2198-08-11 22:06:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),74.0,100.0,ml,FinishedRunning,Piperacillin,Piperacillin,1,metavision
3,200033.0,2198-08-12 00:24:00,2198-08-12 00:25:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),74.0,200.0,ml,Rewritten,Ciprofloxacin,Ciprofloxacin,1,metavision
4,200033.0,2198-08-12 00:25:00,2198-08-12 00:26:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),74.0,200.0,ml,FinishedRunning,Ciprofloxacin,Ciprofloxacin,1,metavision


In [25]:
# create flag variable if antibiotics given to patient
master['antibiotics_flag'] = np.where(master['icustay_id'].isin(antibiotics['icustay_id']),1,0)
master['antibiotics_flag'].value_counts()

0    2883830
1     851325
Name: antibiotics_flag, dtype: int64

In [26]:
# load in pv_mechvent dataset
mechvent = pd.read_csv('mimic_data/pv_mechvent.csv')
mechvent.head()

,icustay_id,charttime,starttime,endtime,duration_hours,ventnum,minutevolume,settidalvolume,obstidalvolume,sponttidalvolume,setpeep,totalpeep,pressurehighaprv,pressurelowaprv,timehighaprv,timelowaprv,meanairwaypressure,peakinsppressure,neginspforce,insptime,plateaupressure
0,200003,2199-08-03 18:00:00,2199-08-03 18:00:00,2199-08-07 13:00:00,91.0,1.0,12.6,600.0,582.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,12.0,26.0,NaN,NaN,19.0
1,200003,2199-08-03 19:00:00,2199-08-03 18:00:00,2199-08-07 13:00:00,91.0,1.0,11.2,600.0,610.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,15.0,26.0,NaN,NaN,22.0
2,200003,2199-08-03 23:00:00,2199-08-03 18:00:00,2199-08-07 13:00:00,91.0,1.0,10.4,600.0,574.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,15.0,26.0,NaN,NaN,24.0
3,200003,2199-08-04 03:00:00,2199-08-03 18:00:00,2199-08-07 13:00:00,91.0,1.0,9.8,600.0,613.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,15.0,27.0,NaN,NaN,22.0
4,200003,2199-08-04 04:00:00,2199-08-03 18:00:00,2199-08-07 13:00:00,91.0,1.0,18.0,600.0,575.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,16.0,28.0,NaN,NaN,NaN


In [27]:
# create flag variable if patient on mechanical ventilation 
master['mechvent_flag'] = np.where(master['icustay_id'].isin(mechvent['icustay_id']),1,0)
master['mechvent_flag'].value_counts()

1    2489247
0    1245908
Name: mechvent_flag, dtype: int64

In [28]:
# load in vasopressor dataset
vasopressors = pd.read_csv('mimic_data/vasopressors.csv')
vasopressors.head()

,icustay_id,starttime,endtime,norepinephrine_rate,norepinephrine_amount,epinephrine_rate,epinephrine_amount,dopamine_rate,dopamine_amount,dobutamine_rate,dobutamine_amount
0,200024.0,2127-03-03 16:15:00,2127-03-03 16:45:00,0.300517,0.638298,NaN,NaN,NaN,NaN,NaN,NaN
1,200024.0,2127-03-03 16:17:00,2127-03-03 20:35:00,NaN,NaN,NaN,NaN,20.03446,365.957468,NaN,NaN
2,200024.0,2127-03-03 16:45:00,2127-03-03 17:15:00,0.200345,0.425532,NaN,NaN,NaN,NaN,NaN,NaN
3,200024.0,2127-03-03 17:15:00,2127-03-03 20:30:00,0.502390,6.936000,NaN,NaN,NaN,NaN,NaN,NaN
4,200028.0,2133-10-29 17:49:00,2133-10-29 18:11:00,0.060011,0.110901,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# create flag variable if patient was given vasopressors --> indicates chronic organ failure (attempting to raise blood pressure)
master['vasopressor_flag'] = np.where(master['icustay_id'].isin(vasopressors['icustay_id']),1,0)
master['vasopressor_flag'].value_counts()

0    2886884
1     848271
Name: vasopressor_flag, dtype: int64

In [30]:
master.head()

,icustay_id,hadm_id,subject_id,intime,outtime,starttime,endtime,hr,dy,gender,dob,age_years,los,icu_expire_flag,ttd_days,gcs,endotrachflag,spo2,heartrate,meanarterialpressure,urineoutput,positiveculture,antibiotics_flag,mechvent_flag,vasopressor_flag
0,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 19:06:12,2181-11-24 20:06:12,-24.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 20:06:12,2181-11-24 21:06:12,-23.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 21:06:12,2181-11-24 22:06:12,-22.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 22:06:12,2181-11-24 23:06:12,-21.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,200001,152234,55973,2181-11-25 19:06:12,2181-11-28 20:59:25,2181-11-24 23:06:12,2181-11-25 00:06:12,-20.0,0.0,F,2120-10-31 00:00:00,61.0,3.0786,0,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
